In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
# import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.stem.wordnet import WordNetLemmatizer

lmtzr= WordNetLemmatizer()


For this challenge, you will need to choose a corpus of data from nltk or another source that includes categories you can predict and create an analysis pipeline that includes the following steps:

- 1. Data cleaning / processing / language parsing
- 2. Create features using two different NLP methods: For example, BoW vs tf-idf.
- 3. Use the features to fit supervised learning models for each feature set to predict the category outcomes.
- 4. Assess your models using cross-validation and determine whether one model performed better.
- 5. Pick one of the models and try to increase accuracy by at least 5 percentage points.


Write up your report in a Jupyter notebook. Be sure to explicitly justify the choices you make throughout, and submit it below.

## load and clean data

In [2]:
# nltk.download()

In [3]:
from nltk.corpus import inaugural

In [4]:
inaugural.fileids()

['1789-Washington.txt',
 '1793-Washington.txt',
 '1797-Adams.txt',
 '1801-Jefferson.txt',
 '1805-Jefferson.txt',
 '1809-Madison.txt',
 '1813-Madison.txt',
 '1817-Monroe.txt',
 '1821-Monroe.txt',
 '1825-Adams.txt',
 '1829-Jackson.txt',
 '1833-Jackson.txt',
 '1837-VanBuren.txt',
 '1841-Harrison.txt',
 '1845-Polk.txt',
 '1849-Taylor.txt',
 '1853-Pierce.txt',
 '1857-Buchanan.txt',
 '1861-Lincoln.txt',
 '1865-Lincoln.txt',
 '1869-Grant.txt',
 '1873-Grant.txt',
 '1877-Hayes.txt',
 '1881-Garfield.txt',
 '1885-Cleveland.txt',
 '1889-Harrison.txt',
 '1893-Cleveland.txt',
 '1897-McKinley.txt',
 '1901-McKinley.txt',
 '1905-Roosevelt.txt',
 '1909-Taft.txt',
 '1913-Wilson.txt',
 '1917-Wilson.txt',
 '1921-Harding.txt',
 '1925-Coolidge.txt',
 '1929-Hoover.txt',
 '1933-Roosevelt.txt',
 '1937-Roosevelt.txt',
 '1941-Roosevelt.txt',
 '1945-Roosevelt.txt',
 '1949-Truman.txt',
 '1953-Eisenhower.txt',
 '1957-Eisenhower.txt',
 '1961-Kennedy.txt',
 '1965-Johnson.txt',
 '1969-Nixon.txt',
 '1973-Nixon.txt',
 '1

For this, I am going to compare the speech of Bish 2005, to that of Obama 2009.


In [5]:
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    

In [6]:
bush = inaugural.raw('2005-Bush.txt')
obama = inaugural.raw('2009-Obama.txt')

In [7]:
bush = bush.lower()
obama = obama.lower()

In [8]:
bush = text_cleaner(bush)
obama = text_cleaner(obama)

In [9]:
sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
bush_sents = sent_tokenizer.tokenize(bush)
obama_sents = sent_tokenizer.tokenize(obama)

In [10]:
bush_sents = [[sent, "bush"] for sent in bush_sents]
obama_sents = [[sent, "obama"] for sent in obama_sents]

sentences = pd.DataFrame(bush_sents + obama_sents) 
sentences.head()

,0,1
0,"vice president cheney, mr. chief justice, pres...",bush
1,"i am grateful for the honor of this hour, mind...",bush
2,"at this second gathering, our duties are defin...",bush
3,"for a half a century, america defended our own...",bush
4,after the shipwreck of communism came years of...,bush


## Bag of words

In [11]:
import string 

def bag_of_words(text, include_stop=False):
    tokenize = word_tokenize(text)
    # Build a list of words.
    # Strip out punctuation and, optionally, stop words.
    words = []
    my_stopwords = stopwords.words('english')
    for token in tokenize:
        if (token not in my_stopwords or include_stop) and (token not in string.punctuation):
            words.append(lmtzr.lemmatize(token))
            
    # Build and return a Counter object containing word counts.
    return [item[0] for item in Counter(words).most_common(2000)]

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words, include_stop = False):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    my_stopwords = stopwords.words('english')
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        list_of_tokens = word_tokenize(sentence)
        words = []
        
        for token in list_of_tokens:
            if (token not in my_stopwords or include_stop) and (token not in string.punctuation)and lmtzr.lemmatize(token) in common_words:
                words.append(lmtzr.lemmatize(token))   
            # Populate the row with word counts.
        for word in words: 
            df.loc[i, word] += 1
                #df.loc[i, :] = 

        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
    
    return df



In [12]:
# Set up the bags.
bushwords = bag_of_words(bush)
obamawords = bag_of_words(obama)

# Combine bags to create a set of unique words.
common_words = set(bushwords + obamawords)

In [13]:
common_words 

{'skill',
 'counsel',
 'united',
 'strive',
 'understood',
 'unwilling',
 'edge',
 'alone',
 'smoke',
 'held',
 'respect',
 'fulfill',
 'generosity',
 'slaughtering',
 'wield',
 'oath',
 'cost',
 'greater',
 'culture',
 'blame',
 'power',
 'collective',
 'serve',
 'dissent',
 'clarify',
 'mercy',
 'toiled',
 'narrow',
 'political',
 'alarmed',
 'flourish',
 'possibility',
 'sermon',
 'fulfilled',
 'ally',
 'car',
 'ill',
 'warms',
 'search',
 'normandy',
 'protecting',
 'precisely',
 'age',
 'surely',
 'america¡¦s',
 'concentrated',
 'family',
 'lessen',
 'gather',
 'new',
 'obligation',
 'always',
 'remained',
 'saw',
 'satisfying',
 'earned',
 'powerful',
 'stained',
 'whisper',
 'part',
 'prudent',
 'depth',
 'willing',
 'everywhere',
 'helping',
 'relies',
 'vulnerability¡xand',
 'hour',
 'member',
 'rely',
 'promote',
 'language',
 'document',
 'achievement',
 'capacity',
 'much',
 'tirelessly',
 'determination',
 'best',
 'measure',
 'murder¡xviolence',
 'imagine',
 'congress',
 

In [14]:
word_counts = bow_features(sentences, common_words)
word_counts.head(100)

Processing row 0


,skill,counsel,united,strive,understood,unwilling,edge,alone,smoke,held,...,promise,first,segregation,purpose,done,life,chance,timeless,text_sentence,text_source
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"vice president cheney, mr. chief justice, pres...",bush
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"i am grateful for the honor of this hour, mind...",bush
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"at this second gathering, our duties are defin...",bush
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"for a half a century, america defended our own...",bush
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,after the shipwreck of communism came years of...,bush
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,we have seen our vulnerability¡xand we have se...,bush
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,for as long as whole regions of the world simm...,bush
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,there is only one force of history that can br...,bush
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"we are led, by events and common sense, to one...",bush
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,the best hope for peace in our world is the ex...,bush


In [15]:
sentences.iloc[24,0]


'america¡¦s influence is not unlimited, but fortunately for the oppressed, america¡¦s influence is considerable, and we will use it confidently in freedom¡¦s cause.'

going to add two feature, sentence length and count of punctuation

In [16]:

word_counts["sentence_length"] = word_counts['text_sentence'].apply(lambda x: len(word_tokenize(x)))
word_counts['punc_count'] = word_counts['text_sentence'].apply(lambda x: sum([token in string.punctuation for token in word_tokenize(x)]))


In [17]:
word_counts

,skill,counsel,united,strive,understood,unwilling,edge,alone,smoke,held,...,segregation,purpose,done,life,chance,timeless,text_sentence,text_source,sentence_length,punc_count
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"vice president cheney, mr. chief justice, pres...",bush,64,13
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"i am grateful for the honor of this hour, mind...",bush,35,3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"at this second gathering, our duties are defin...",bush,25,3
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"for a half a century, america defended our own...",bush,18,2
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,after the shipwreck of communism came years of...,bush,26,3
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,we have seen our vulnerability¡xand we have se...,bush,12,1
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,for as long as whole regions of the world simm...,bush,44,4
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,there is only one force of history that can br...,bush,43,4
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"we are led, by events and common sense, to one...",bush,32,4
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,the best hope for peace in our world is the ex...,bush,18,1


# bow modeling
## RF

In [18]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))


BoW_RFC_score = cross_val_score(rfc, X, Y, cv=5)
print('\nEach Cross Validated Accuracy: \n', BoW_RFC_score)
print("\nOverall Random Forest Classifier Accuracy: %0.2f (+/- %0.2f)\n" % (BoW_RFC_score.mean(), BoW_RFC_score.std() * 2))


Training set score: 0.959349593496

Test set score: 0.634146341463

Each Cross Validated Accuracy: 
 [ 0.70731707  0.65853659  0.51219512  0.82926829  0.7804878 ]

Overall Random Forest Classifier Accuracy: 0.70 (+/- 0.22)



In [19]:
#going to try something with grid search
from sklearn.model_selection import GridSearchCV
param_search = [{'n_estimators': [2,3,5,10,15,20,100,200],
          'max_depth': [3,4,5,6,8,10,15],
          'min_samples_split':[2,3,5,6,8,10,15,20]}]


# Initialize and fit the model.
rfc_grid = GridSearchCV(ensemble.RandomForestClassifier(), param_grid=param_search,cv=5)

rfc_grid.fit(X,Y)

print('Best Score: ', rfc_grid.best_score_)
print('Best Parameters: ', rfc_grid.best_params_)

Best Score:  0.717073170732
Best Parameters:  {'max_depth': 15, 'min_samples_split': 5, 'n_estimators': 20}


In [36]:
rfc = ensemble.RandomForestClassifier(n_estimators=20,max_depth=15,min_samples_split=5)
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))


BoW_RFC_score = cross_val_score(rfc, X, Y, cv=5)
print('\nEach Cross Validated Accuracy: \n', BoW_RFC_score)
print("\nOverall Random Forest Classifier Accuracy: %0.2f (+/- %0.2f)\n" % (BoW_RFC_score.mean(), BoW_RFC_score.std() * 2))

Training set score: 0.967479674797

Test set score: 0.719512195122

Each Cross Validated Accuracy: 
 [ 0.65853659  0.58536585  0.63414634  0.75609756  0.75609756]

Overall Random Forest Classifier Accuracy: 0.68 (+/- 0.14)



## LOG

In [21]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))


BoW_lr_score = cross_val_score(lr, X, Y, cv=5)
print('\nEach Cross Validated Accuracy: \n', BoW_lr_score)
print("\nOverall Logistic Regression Classifier Accuracy: %0.2f (+/- %0.2f)\n" % (BoW_lr_score.mean(), BoW_lr_score.std() * 2))


(123, 1159) (123,)
Training set score: 1.0

Test set score: 0.829268292683

Each Cross Validated Accuracy: 
 [ 0.75609756  0.68292683  0.65853659  0.7804878   0.70731707]

Overall Logistic Regression Classifier Accuracy: 0.72 (+/- 0.09)



## GB

In [22]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))


BoW_clf_score = cross_val_score(clf, X, Y, cv=5)
print('\nEach Cross Validated Accuracy: \n', BoW_clf_score)
print("\nOverall Gradient Boosting Classifier Accuracy: %0.2f (+/- %0.2f)\n" % (BoW_clf_score.mean(), BoW_clf_score.std() * 2))

Training set score: 1.0

Test set score: 0.756097560976

Each Cross Validated Accuracy: 
 [ 0.6097561   0.63414634  0.63414634  0.75609756  0.73170732]

Overall Gradient Boosting Classifier Accuracy: 0.67 (+/- 0.12)



clearly some overfitting going on so I am going to use some vectorizes and also some normalization methods to try and correct this. i will also do this in convert with the tfidf methods

## vectorize tdidf and bow

going to use the sentences df we originally created

In [37]:
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    

In [38]:
bush = inaugural.raw('2005-Bush.txt')
obama = inaugural.raw('2009-Obama.txt')

In [39]:
bush = bush.lower()
obama = obama.lower()

In [40]:
bush = text_cleaner(bush)
obama = text_cleaner(obama)

In [41]:
sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
bush_sents = sent_tokenizer.tokenize(bush)
obama_sents = sent_tokenizer.tokenize(obama)

In [42]:
new_bush = []
for sentence in bush_sents: 
    list_of_tokens = word_tokenize(sentence)
    bush_lemmas = []
#     print(list_of_tokens)
    for token in list_of_tokens: 
        if (token not in string.punctuation): 
            bush_lemmas.append(lmtzr.lemmatize(token))
    new_bush.append(" ".join(bush_lemmas))
    
new_obama = []
for sentence in obama_sents: 
    list_of_tokens = word_tokenize(sentence)
    obama_lemmas = []
#     print(list_of_tokens)
    for token in list_of_tokens: 
        if (token not in string.punctuation): 
            obama_lemmas.append(lmtzr.lemmatize(token))
    new_obama.append(" ".join(obama_lemmas))
            

In [43]:
bush_sentences = [[sent, "bush"] for sent in new_bush]
obama_sentences= [[sent, "obama"] for sent in new_obama]

sentences = pd.DataFrame(bush_sentences + obama_sentences) 
sentences.head()

,0,1
0,vice president cheney mr. chief justice presid...,bush
1,i am grateful for the honor of this hour mindf...,bush
2,at this second gathering our duty are defined ...,bush
3,for a half a century america defended our own ...,bush
4,after the shipwreck of communism came year of ...,bush


In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

Y = sentences.iloc[:,1]
X = sentences.iloc[:,0]

# X_train, X_test, y_train, y_test = train_test_split(X, 
#                                                     Y,
#                                                     test_size=0.4,
#                                                     random_state=0)

tfidf_vectorizer = TfidfVectorizer(max_df=0.5, # drop words that occur in more than 80% of paragraphs
                             min_df=2, # only use words that appear at least twice
                             stop_words='english', # mostly redundant, but it's ok 
                             lowercase=True, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Starting with L2 regularization
                             smooth_idf=True #Prevents divide-by-zero errors
                            )

bow_vectorizer = TfidfVectorizer(max_df=0.5, # drop words that occur in more than 80% of paragraphs
                             min_df=2, # only use words that appear at least twice
                             stop_words='english', # mostly redundant, but it's ok 
                             lowercase=True, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=False,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Starting with L2 regularization
                             smooth_idf=False #Prevents divide-by-zero errors
                            )

tfidf = tfidf_vectorizer.fit_transform(X)
bow = bow_vectorizer.fit_transform(X)

# X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf= train_test_split(tfidf, Y, test_size=0.4, random_state=0)
# X_train_bow, X_test_bow, y_train_bow, y_test_bow= train_test_split(bow, Y, test_size=0.4, random_state=0)

# SVD!
svd= TruncatedSVD(n_components = 10)
lsa = make_pipeline(svd, Normalizer(copy=False))
# Run SVD on the training data, then project the training data.
lsa_tfidf= lsa.fit_transform(tfidf)
lsa_bow= lsa.fit_transform(bow)



In [80]:
bow

<205x329 sparse matrix of type '<class 'numpy.float64'>'
	with 1207 stored elements in Compressed Sparse Row format>

In [81]:
#whats the best way to use the lsa in the model below? 

## try rfc

In [82]:
rfc = ensemble.RandomForestClassifier(n_estimators=10)


BoW_RFC_score = cross_val_score(rfc, lsa_bow, Y, cv=5)
print('\nEach BOW Cross Validated Accuracy: \n', BoW_RFC_score)
print("\nOverall Random Forest Classifier Accuracy: %0.2f (+/- %0.2f)\n" % (BoW_RFC_score.mean(), BoW_RFC_score.std() * 2))

TFIDF_RFC_score = cross_val_score(rfc, lsa_tfidf, Y, cv=5)
print('\nEach TFIDF Cross Validated Accuracy: \n', TFIDF_RFC_score)
print("\nOverall Random Forest Classifier Accuracy: %0.2f (+/- %0.2f)\n" % (TFIDF_RFC_score.mean(), TFIDF_RFC_score.std() * 2))


Each BOW Cross Validated Accuracy: 
 [ 0.70731707  0.68292683  0.63414634  0.58536585  0.65853659]

Overall Random Forest Classifier Accuracy: 0.65 (+/- 0.08)


Each TFIDF Cross Validated Accuracy: 
 [ 0.65853659  0.65853659  0.6097561   0.65853659  0.73170732]

Overall Random Forest Classifier Accuracy: 0.66 (+/- 0.08)



## LOG

In [87]:
lr = LogisticRegression(penalty='l1')


BoW_lr_score = cross_val_score(lr, lsa_bow, Y, cv=5)
print('\nEach Cross Validated Accuracy: \n', BoW_lr_score)
print("\nOverall Random Forest Classifier Accuracy: %0.2f (+/- %0.2f)\n" % (BoW_lr_score.mean(), BoW_lr_score.std() * 2))

TFIDF_lr_score = cross_val_score(lr, lsa_tfidf, Y, cv=10)
print('\nEach Cross Validated Accuracy: \n', TFIDF_lr_score)
print("\nOverall Random Forest Classifier Accuracy: %0.2f (+/- %0.2f)\n" % (TFIDF_lr_score.mean(), TFIDF_lr_score.std() * 2))





Each Cross Validated Accuracy: 
 [ 0.73170732  0.68292683  0.58536585  0.70731707  0.68292683]

Overall Random Forest Classifier Accuracy: 0.68 (+/- 0.10)


Each Cross Validated Accuracy: 
 [ 0.66666667  0.61904762  0.61904762  0.85714286  0.66666667  0.45        0.7
  0.65        0.9         0.75      ]

Overall Random Forest Classifier Accuracy: 0.69 (+/- 0.24)



## GB

In [83]:

clf = ensemble.GradientBoostingClassifier()

BoW_clf_score = cross_val_score(clf, lsa_bow, Y, cv=5)
print('\nEach Cross Validated Accuracy: \n', BoW_clf_score)
print("\nOverall Random Forest Classifier Accuracy: %0.2f (+/- %0.2f)\n" % (BoW_clf_score.mean(), BoW_clf_score.std() * 2))

TFIDF_clf_score = cross_val_score(clf, lsa_tfidf, Y, cv=5)
print('\nEach Cross Validated Accuracy: \n', TFIDF_clf_score)
print("\nOverall Random Forest Classifier Accuracy: %0.2f (+/- %0.2f)\n" % (TFIDF_clf_score.mean(), TFIDF_clf_score.std() * 2))


Each Cross Validated Accuracy: 
 [ 0.68292683  0.7804878   0.68292683  0.68292683  0.70731707]

Overall Random Forest Classifier Accuracy: 0.71 (+/- 0.08)


Each Cross Validated Accuracy: 
 [ 0.63414634  0.7804878   0.63414634  0.73170732  0.75609756]

Overall Random Forest Classifier Accuracy: 0.71 (+/- 0.12)

